# Лабораторная работа 4

Tensorflow 2.x

1) Подготовка данных

2) Использование Keras Model API

3) Использование Keras Sequential + Functional API

https://www.tensorflow.org/tutorials

Для выполнения лабораторной работы необходимо установить tensorflow версии 2.0 или выше .

Рекомендуется использовать возможности Colab'а по обучению моделей на GPU.



In [6]:
import os
import tensorflow as tf
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt

device = "/GPU:0" if tf.config.list_physical_devices('GPU') else "/CPU:0"

%matplotlib inline

# Подготовка данных
Загрузите набор данных из предыдущей лабораторной работы. 

In [2]:
def load_cifar10(num_training=49000, num_validation=1000, num_test=10000):
    """
    Fetch the CIFAR-10 dataset from the web and perform preprocessing to prepare
    it for the two-layer neural net classifier. These are the same steps as
    we used for the SVM, but condensed to a single function.
    """
    # Load the raw CIFAR-10 dataset and use appropriate data types and shapes
    cifar10 = tf.keras.datasets.cifar10.load_data()
    (X_train, y_train), (X_test, y_test) = cifar10
    X_train = np.asarray(X_train, dtype=np.float32)
    y_train = np.asarray(y_train, dtype=np.int32).flatten()
    X_test = np.asarray(X_test, dtype=np.float32)
    y_test = np.asarray(y_test, dtype=np.int32).flatten()

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    # Normalize the data: subtract the mean pixel and divide by std
    mean_pixel = X_train.mean(axis=(0, 1, 2), keepdims=True)
    std_pixel = X_train.std(axis=(0, 1, 2), keepdims=True)
    X_train = (X_train - mean_pixel) / std_pixel
    X_val = (X_val - mean_pixel) / std_pixel
    X_test = (X_test - mean_pixel) / std_pixel

    return X_train, y_train, X_val, y_val, X_test, y_test

# If there are errors with SSL downloading involving self-signed certificates,
# it may be that your Python version was recently installed on the current machine.
# See: https://github.com/tensorflow/tensorflow/issues/10779
# To fix, run the command: /Applications/Python\ 3.7/Install\ Certificates.command
#   ...replacing paths as necessary.

# Invoke the above function to get our data.
NHW = (0, 1, 2)
X_train, y_train, X_val, y_val, X_test, y_test = load_cifar10()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 22s 0us/step
Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000,) int32
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


In [3]:
class Dataset(object):
    def __init__(self, X, y, batch_size, shuffle=False):
        """
        Construct a Dataset object to iterate over data X and labels y
        
        Inputs:
        - X: Numpy array of data, of any shape
        - y: Numpy array of labels, of any shape but with y.shape[0] == X.shape[0]
        - batch_size: Integer giving number of elements per minibatch
        - shuffle: (optional) Boolean, whether to shuffle the data on each epoch
        """
        assert X.shape[0] == y.shape[0], 'Got different numbers of data and labels'
        self.X, self.y = X, y
        self.batch_size, self.shuffle = batch_size, shuffle

    def __iter__(self):
        N, B = self.X.shape[0], self.batch_size
        idxs = np.arange(N)
        if self.shuffle:
            np.random.shuffle(idxs)
        return iter((self.X[i:i+B], self.y[i:i+B]) for i in range(0, N, B))


train_dset = Dataset(X_train, y_train, batch_size=64, shuffle=True)
val_dset = Dataset(X_val, y_val, batch_size=64, shuffle=False)
test_dset = Dataset(X_test, y_test, batch_size=64)

In [4]:
# We can iterate through a dataset like this:
for t, (x, y) in enumerate(train_dset):
    print(t, x.shape, y.shape)
    if t > 5: break

0 (64, 32, 32, 3) (64,)
1 (64, 32, 32, 3) (64,)
2 (64, 32, 32, 3) (64,)
3 (64, 32, 32, 3) (64,)
4 (64, 32, 32, 3) (64,)
5 (64, 32, 32, 3) (64,)
6 (64, 32, 32, 3) (64,)


#  Keras Model Subclassing API


Для реализации собственной модели с помощью Keras Model Subclassing API необходимо выполнить следующие шаги:

1) Определить новый класс, который является наследником tf.keras.Model.

2) В методе __init__() определить все необходимые слои из модуля tf.keras.layer

3) Реализовать прямой проход в методе call() на основе слоев, объявленных в __init__()

Ниже приведен пример использования keras API для определения двухслойной полносвязной сети. 

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras

In [7]:
class TwoLayerFC(tf.keras.Model):
    def __init__(self, hidden_size, num_classes):
        super(TwoLayerFC, self).__init__()        
        initializer = tf.initializers.VarianceScaling(scale=2.0)
        self.fc1 = tf.keras.layers.Dense(hidden_size, activation='relu',
                                   kernel_initializer=initializer)
        self.fc2 = tf.keras.layers.Dense(num_classes, activation='softmax',
                                   kernel_initializer=initializer)
        self.flatten = tf.keras.layers.Flatten()
    
    def call(self, x, training=False):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x


def test_TwoLayerFC():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    x = tf.zeros((64, input_size))
    model = TwoLayerFC(hidden_size, num_classes)
    with tf.device(device):
        scores = model(x)
        print(scores.shape)
        
test_TwoLayerFC()

(64, 10)


Реализуйте трехслойную CNN для вашей задачи классификации. 

Архитектура сети:
    
1. Сверточный слой (5 x 5 kernels, zero-padding = 'same')
2. Функция активации ReLU 
3. Сверточный слой (3 x 3 kernels, zero-padding = 'same')
4. Функция активации ReLU 
5. Полносвязный слой 
6. Функция активации Softmax 

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Conv2D

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dense

In [8]:
class ThreeLayerConvNet(tf.keras.Model):
    def __init__(self, channel_1, channel_2, num_classes):
        super(ThreeLayerConvNet, self).__init__()
        ########################################################################
        # TODO: Implement the __init__ method for a three-layer ConvNet. You   #
        # should instantiate layer objects to be used in the forward pass.     #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        initializer = tf.initializers.VarianceScaling(scale=2.0)

        # Conv2D с ядром 5x5, padding='same'
        self.conv1 = tf.keras.layers.Conv2D(
            filters=channel_1,
            kernel_size=(5, 5),
            padding='same',
            activation='relu',
            kernel_initializer=initializer
        )

        # Conv2D с ядром 3x3, padding='same'
        self.conv2 = tf.keras.layers.Conv2D(
            filters=channel_2,
            kernel_size=(3, 3),
            padding='same',
            activation='relu',
            kernel_initializer=initializer
        )

        # Преобразование feature map → вектор
        self.flatten = tf.keras.layers.Flatten()

        # Полносвязный слой + softmax
        self.fc = tf.keras.layers.Dense(
            num_classes,
            activation='softmax',
            kernel_initializer=initializer
        )

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                           END OF YOUR CODE                           #
        ########################################################################
        
    def call(self, x, training=False):
        scores = None
        ########################################################################
        # TODO: Implement the forward pass for a three-layer ConvNet. You      #
        # should use the layer objects defined in the __init__ method.         #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        x = self.conv1(x)      # conv 5x5 + ReLU
        x = self.conv2(x)      # conv 3x3 + ReLU
        x = self.flatten(x)    # разворачиваем карты признаков
        scores = self.fc(x)    # Dense + softmax

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                           END OF YOUR CODE                           #
        ########################################################################        
        return scores

In [9]:
def test_ThreeLayerConvNet():    
    channel_1, channel_2, num_classes = 12, 8, 10
    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)
    with tf.device(device):
        x = tf.zeros((64, 3, 32, 32))
        scores = model(x)
        print(scores.shape)

test_ThreeLayerConvNet()

(64, 10)


Пример реализации процесса обучения:

In [12]:
def train_part34(model_init_fn, optimizer_init_fn, num_epochs=1, is_training=False):
    """
    Simple training loop for use with models defined using tf.keras. It trains
    a model for one epoch on the CIFAR-10 training set and periodically checks
    accuracy on the CIFAR-10 validation set.
    
    Inputs:
    - model_init_fn: A function that takes no parameters; when called it
      constructs the model we want to train: model = model_init_fn()
    - optimizer_init_fn: A function which takes no parameters; when called it
      constructs the Optimizer object we will use to optimize the model:
      optimizer = optimizer_init_fn()
    - num_epochs: The number of epochs to train for
    
    Returns: Nothing, but prints progress during trainingn
    """    
    with tf.device(device):

        
        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
        
        model = model_init_fn()
        optimizer = optimizer_init_fn()
        
        train_loss = tf.keras.metrics.Mean(name='train_loss')
        train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
    
        val_loss = tf.keras.metrics.Mean(name='val_loss')
        val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')
        
        t = 0
        for epoch in range(num_epochs):
            
            # Reset the metrics - https://www.tensorflow.org/alpha/guide/migration_guide#new-style_metrics
            train_loss.reset_state()
            train_accuracy.reset_state()
            
            for x_np, y_np in train_dset:
                with tf.GradientTape() as tape:
                    
                    # Use the model function to build the forward pass.
                    scores = model(x_np, training=is_training)
                    loss = loss_fn(y_np, scores)
      
                    gradients = tape.gradient(loss, model.trainable_variables)
                    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
                    
                    # Update the metrics
                    train_loss.update_state(loss)
                    train_accuracy.update_state(y_np, scores)
                    
                    if t % print_every == 0:
                        val_loss.reset_state()
                        val_accuracy.reset_state()
                        for test_x, test_y in val_dset:
                            # During validation at end of epoch, training set to False
                            prediction = model(test_x, training=False)
                            t_loss = loss_fn(test_y, prediction)

                            val_loss.update_state(t_loss)
                            val_accuracy.update_state(test_y, prediction)
                        
                        template = 'Iteration {}, Epoch {}, Loss: {}, Accuracy: {}, Val Loss: {}, Val Accuracy: {}'
                        print (template.format(t, epoch+1,
                                             train_loss.result(),
                                             train_accuracy.result()*100,
                                             val_loss.result(),
                                             val_accuracy.result()*100))
                    t += 1

In [14]:
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn():
    return TwoLayerFC(hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

print_every = 100
train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.909259796142578, Accuracy: 14.0625, Val Loss: 2.8645734786987305, Val Accuracy: 10.199999809265137
Iteration 100, Epoch 1, Loss: 2.2256886959075928, Accuracy: 28.743812561035156, Val Loss: 1.9273066520690918, Val Accuracy: 36.5
Iteration 200, Epoch 1, Loss: 2.0685300827026367, Accuracy: 32.361629486083984, Val Loss: 1.883780837059021, Val Accuracy: 40.400001525878906
Iteration 300, Epoch 1, Loss: 1.9945646524429321, Accuracy: 34.073917388916016, Val Loss: 1.8746986389160156, Val Accuracy: 37.599998474121094
Iteration 400, Epoch 1, Loss: 1.9266550540924072, Accuracy: 35.9413948059082, Val Loss: 1.7146981954574585, Val Accuracy: 43.0
Iteration 500, Epoch 1, Loss: 1.8835428953170776, Accuracy: 37.02594757080078, Val Loss: 1.6765518188476562, Val Accuracy: 42.69999694824219
Iteration 600, Epoch 1, Loss: 1.8529891967773438, Accuracy: 38.00696563720703, Val Loss: 1.6880568265914917, Val Accuracy: 42.89999771118164
Iteration 700, Epoch 1, Loss: 1.8266627788543701

Обучите трехслойную CNN. В tf.keras.optimizers.SGD укажите Nesterov momentum = 0.9 . 

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/optimizers/SGD

Значение accuracy на валидационной выборке после 1 эпохи обучения должно быть > 50% .

In [15]:
learning_rate = 3e-3
channel_1, channel_2, num_classes = 32, 16, 10

def model_init_fn():
    model = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return model

def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    # SGD с Nesterov momentum = 0.9
    optimizer = tf.keras.optimizers.SGD(
        learning_rate=learning_rate,
        momentum=0.9,
        nesterov=True
    )

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.7724995613098145, Accuracy: 6.25, Val Loss: 2.8599705696105957, Val Accuracy: 8.299999237060547
Iteration 100, Epoch 1, Loss: 1.8473314046859741, Accuracy: 34.173885345458984, Val Loss: 1.597209095954895, Val Accuracy: 45.0
Iteration 200, Epoch 1, Loss: 1.6887739896774292, Accuracy: 40.18967819213867, Val Loss: 1.4742238521575928, Val Accuracy: 47.900001525878906
Iteration 300, Epoch 1, Loss: 1.60145902633667, Accuracy: 43.35029220581055, Val Loss: 1.384963035583496, Val Accuracy: 52.70000076293945
Iteration 400, Epoch 1, Loss: 1.5335887670516968, Accuracy: 45.717735290527344, Val Loss: 1.319648027420044, Val Accuracy: 53.70000076293945
Iteration 500, Epoch 1, Loss: 1.485134482383728, Accuracy: 47.40830993652344, Val Loss: 1.2847833633422852, Val Accuracy: 55.900001525878906
Iteration 600, Epoch 1, Loss: 1.4534255266189575, Accuracy: 48.45050048828125, Val Loss: 1.2449060678482056, Val Accuracy: 57.20000457763672
Iteration 700, Epoch 1, Loss: 1.42419576644

# Использование Keras Sequential API для реализации последовательных моделей.

Пример для полносвязной сети:

In [16]:
learning_rate = 1e-2

def model_init_fn():
    input_shape = (32, 32, 3)
    hidden_layer_size, num_classes = 4000, 10
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    layers = [
        tf.keras.layers.Flatten(input_shape=input_shape),
        tf.keras.layers.Dense(hidden_layer_size, activation='relu',
                              kernel_initializer=initializer),
        tf.keras.layers.Dense(num_classes, activation='softmax', 
                              kernel_initializer=initializer),
    ]
    model = tf.keras.Sequential(layers)
    return model

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate) 

train_part34(model_init_fn, optimizer_init_fn)

c:\Users\ruslk\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Iteration 0, Epoch 1, Loss: 2.9988365173339844, Accuracy: 10.9375, Val Loss: 2.8405532836914062, Val Accuracy: 12.5
Iteration 100, Epoch 1, Loss: 2.2333595752716064, Accuracy: 27.985767364501953, Val Loss: 1.923144817352295, Val Accuracy: 36.89999771118164
Iteration 200, Epoch 1, Loss: 2.0760116577148438, Accuracy: 31.708642959594727, Val Loss: 1.8855068683624268, Val Accuracy: 38.10000228881836
Iteration 300, Epoch 1, Loss: 2.001803159713745, Accuracy: 33.741695404052734, Val Loss: 1.9236003160476685, Val Accuracy: 36.19999694824219
Iteration 400, Epoch 1, Loss: 1.9331634044647217, Accuracy: 35.680328369140625, Val Loss: 1.7629584074020386, Val Accuracy: 42.0
Iteration 500, Epoch 1, Loss: 1.8885114192962646, Accuracy: 36.79203796386719, Val Loss: 1.6843791007995605, Val Accuracy: 41.900001525878906
Iteration 600, Epoch 1, Loss: 1.857667088508606, Accuracy: 37.69758605957031, Val Loss: 1.7173736095428467, Val Accuracy: 41.80000305175781
Iteration 700, Epoch 1, Loss: 1.8320178985595703,

Альтернативный менее гибкий способ обучения:

In [17]:
model = model_init_fn()
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)

766/766 ━━━━━━━━━━━━━━━━━━━━ 21s 27ms/step - loss: 2.0376 - sparse_categorical_accuracy: 0.3310 - val_loss: 1.7370 - val_sparse_categorical_accuracy: 0.3960
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 1.7047 - sparse_categorical_accuracy: 0.4086


[1.7122572660446167, 0.41110000014305115]

Перепишите реализацию трехслойной CNN с помощью tf.keras.Sequential API . Обучите модель двумя способами.

In [18]:
def model_init_fn():
    model = None
    ############################################################################
    # TODO: Construct a three-layer ConvNet using tf.keras.Sequential.         #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    initializer = tf.initializers.VarianceScaling(scale=2.0)

    # CIFAR-10 обычно в формате (N, 32, 32, 3)
    model = tf.keras.Sequential([
        # Conv 5x5, padding='same' + ReLU
        tf.keras.layers.Conv2D(
            filters=channel_1,
            kernel_size=(5, 5),
            padding='same',
            activation='relu',
            kernel_initializer=initializer,
            input_shape=(32, 32, 3)
        ),
        # Conv 3x3, padding='same' + ReLU
        tf.keras.layers.Conv2D(
            filters=channel_2,
            kernel_size=(3, 3),
            padding='same',
            activation='relu',
            kernel_initializer=initializer
        ),
        # Flatten + Dense + Softmax
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(
            num_classes,
            activation='softmax',
            kernel_initializer=initializer
        ),
    ])

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                            END OF YOUR CODE                              #
    ############################################################################
    return model

learning_rate = 5e-4
def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

c:\Users\ruslk\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Iteration 0, Epoch 1, Loss: 3.3625969886779785, Accuracy: 12.5, Val Loss: 5.1657538414001465, Val Accuracy: 9.800000190734863
Iteration 100, Epoch 1, Loss: 1.98892343044281, Accuracy: 33.81806945800781, Val Loss: 1.6618074178695679, Val Accuracy: 42.89999771118164
Iteration 200, Epoch 1, Loss: 1.789108395576477, Accuracy: 39.13246536254883, Val Loss: 1.5090497732162476, Val Accuracy: 47.60000228881836
Iteration 300, Epoch 1, Loss: 1.6924915313720703, Accuracy: 41.79817199707031, Val Loss: 1.4210867881774902, Val Accuracy: 49.39999771118164
Iteration 400, Epoch 1, Loss: 1.6138687133789062, Accuracy: 44.02665328979492, Val Loss: 1.3713467121124268, Val Accuracy: 52.20000076293945
Iteration 500, Epoch 1, Loss: 1.561739206314087, Accuracy: 45.484031677246094, Val Loss: 1.3501315116882324, Val Accuracy: 53.500003814697266
Iteration 600, Epoch 1, Loss: 1.526901125907898, Accuracy: 46.6228141784668, Val Loss: 1.2973182201385498, Val Accuracy: 55.5
Iteration 700, Epoch 1, Loss: 1.4969959259033

In [19]:
model = model_init_fn()
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)

766/766 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - loss: 1.8441 - sparse_categorical_accuracy: 0.3732 - val_loss: 1.3228 - val_sparse_categorical_accuracy: 0.5250
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.3350 - sparse_categorical_accuracy: 0.5230


[1.3380333185195923, 0.5238999724388123]

# Использование Keras Functional API

Для реализации более сложных архитектур сети с несколькими входами/выходами, повторным использованием слоев, "остаточными" связями (residual connections) необходимо явно указать входные и выходные тензоры. 

Ниже представлен пример для полносвязной сети. 

In [20]:
def two_layer_fc_functional(input_shape, hidden_size, num_classes):  
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    inputs = tf.keras.Input(shape=input_shape)
    flattened_inputs = tf.keras.layers.Flatten()(inputs)
    fc1_output = tf.keras.layers.Dense(hidden_size, activation='relu',
                                 kernel_initializer=initializer)(flattened_inputs)
    scores = tf.keras.layers.Dense(num_classes, activation='softmax',
                             kernel_initializer=initializer)(fc1_output)

    # Instantiate the model given inputs and outputs.
    model = tf.keras.Model(inputs=inputs, outputs=scores)
    return model

def test_two_layer_fc_functional():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    input_shape = (50,)
    
    x = tf.zeros((64, input_size))
    model = two_layer_fc_functional(input_shape, hidden_size, num_classes)
    
    with tf.device(device):
        scores = model(x)
        print(scores.shape)
        
test_two_layer_fc_functional()

(64, 10)


In [21]:
input_shape = (32, 32, 3)
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn():
    return two_layer_fc_functional(input_shape, hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 3.0616657733917236, Accuracy: 6.25, Val Loss: 2.9997007846832275, Val Accuracy: 12.300000190734863
Iteration 100, Epoch 1, Loss: 2.2502951622009277, Accuracy: 28.46534538269043, Val Loss: 1.8916140794754028, Val Accuracy: 37.5
Iteration 200, Epoch 1, Loss: 2.0879456996917725, Accuracy: 31.96517562866211, Val Loss: 1.8514595031738281, Val Accuracy: 37.900001525878906
Iteration 300, Epoch 1, Loss: 2.0109994411468506, Accuracy: 33.996055603027344, Val Loss: 1.8443447351455688, Val Accuracy: 38.60000228881836
Iteration 400, Epoch 1, Loss: 1.9390240907669067, Accuracy: 35.96477508544922, Val Loss: 1.7335431575775146, Val Accuracy: 41.900001525878906
Iteration 500, Epoch 1, Loss: 1.894154667854309, Accuracy: 36.91679382324219, Val Loss: 1.6443979740142822, Val Accuracy: 44.20000076293945
Iteration 600, Epoch 1, Loss: 1.864894986152649, Accuracy: 37.77298355102539, Val Loss: 1.6814550161361694, Val Accuracy: 40.79999923706055
Iteration 700, Epoch 1, Loss: 1.8389610

Поэкспериментируйте с архитектурой сверточной сети. Для вашего набора данных вам необходимо получить как минимум 70% accuracy на валидационной выборке за 10 эпох обучения. Опишите все эксперименты и сделайте выводы (без выполнения данного пункта работы приниматься не будут). 

Эспериментируйте с архитектурой, гиперпараметрами, функцией потерь, регуляризацией, методом оптимизации.  

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/BatchNormalization#methods https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dropout#methods

In [22]:
class CustomConvNet(tf.keras.Model):
    def __init__(self):
        super(CustomConvNet, self).__init__()
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        initializer = tf.keras.initializers.HeNormal()

        # Block 1: (32x32) -> (16x16)
        self.conv1_1 = tf.keras.layers.Conv2D(
            32, (3, 3), padding='same',
            kernel_initializer=initializer, use_bias=False
        )
        self.bn1_1 = tf.keras.layers.BatchNormalization()

        self.conv1_2 = tf.keras.layers.Conv2D(
            32, (3, 3), padding='same',
            kernel_initializer=initializer, use_bias=False
        )
        self.bn1_2 = tf.keras.layers.BatchNormalization()

        self.pool1 = tf.keras.layers.MaxPool2D(pool_size=(2, 2))
        self.drop1 = tf.keras.layers.Dropout(0.25)

        # Block 2: (16x16) -> (8x8)
        self.conv2_1 = tf.keras.layers.Conv2D(
            64, (3, 3), padding='same',
            kernel_initializer=initializer, use_bias=False
        )
        self.bn2_1 = tf.keras.layers.BatchNormalization()

        self.conv2_2 = tf.keras.layers.Conv2D(
            64, (3, 3), padding='same',
            kernel_initializer=initializer, use_bias=False
        )
        self.bn2_2 = tf.keras.layers.BatchNormalization()

        self.pool2 = tf.keras.layers.MaxPool2D(pool_size=(2, 2))
        self.drop2 = tf.keras.layers.Dropout(0.25)

        # Block 3: (8x8) -> (4x4)
        self.conv3_1 = tf.keras.layers.Conv2D(
            128, (3, 3), padding='same',
            kernel_initializer=initializer, use_bias=False
        )
        self.bn3_1 = tf.keras.layers.BatchNormalization()

        self.pool3 = tf.keras.layers.MaxPool2D(pool_size=(2, 2))
        self.drop3 = tf.keras.layers.Dropout(0.4)

        # Classifier head
        self.flatten = tf.keras.layers.Flatten()
        self.fc1 = tf.keras.layers.Dense(
            256, kernel_initializer=initializer, use_bias=False
        )
        self.bn_fc1 = tf.keras.layers.BatchNormalization()
        self.drop_fc1 = tf.keras.layers.Dropout(0.5)

        # Выход: num_classes = 10, softmax
        self.fc_out = tf.keras.layers.Dense(
            10, activation='softmax'
        )

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
    
    def call(self, input_tensor, training=False):
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        x = input_tensor

        # Block 1
        x = self.conv1_1(x)
        x = self.bn1_1(x, training=training)
        x = tf.nn.relu(x)

        x = self.conv1_2(x)
        x = self.bn1_2(x, training=training)
        x = tf.nn.relu(x)

        x = self.pool1(x)
        x = self.drop1(x, training=training)

        # Block 2
        x = self.conv2_1(x)
        x = self.bn2_1(x, training=training)
        x = tf.nn.relu(x)

        x = self.conv2_2(x)
        x = self.bn2_2(x, training=training)
        x = tf.nn.relu(x)

        x = self.pool2(x)
        x = self.drop2(x, training=training)

        # Block 3
        x = self.conv3_1(x)
        x = self.bn3_1(x, training=training)
        x = tf.nn.relu(x)

        x = self.pool3(x)
        x = self.drop3(x, training=training)

        # Classifier head
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.bn_fc1(x, training=training)
        x = tf.nn.relu(x)
        x = self.drop_fc1(x, training=training)

        x = self.fc_out(x)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
        return x


print_every = 700
num_epochs = 10

model = CustomConvNet()

def model_init_fn():
    return CustomConvNet()

def optimizer_init_fn():
    learning_rate = 1e-3
    return tf.keras.optimizers.Adam(learning_rate) 

train_part34(model_init_fn, optimizer_init_fn, num_epochs=num_epochs, is_training=True)

Iteration 0, Epoch 1, Loss: 3.603207588195801, Accuracy: 4.6875, Val Loss: 4.421055793762207, Val Accuracy: 8.399999618530273
Iteration 700, Epoch 1, Loss: 1.6047779321670532, Accuracy: 42.79823684692383, Val Loss: 1.252152442932129, Val Accuracy: 57.400001525878906
Iteration 1400, Epoch 2, Loss: 1.135211706161499, Accuracy: 59.694881439208984, Val Loss: 0.9000283479690552, Val Accuracy: 68.9000015258789
Iteration 2100, Epoch 3, Loss: 0.9655840992927551, Accuracy: 65.90509033203125, Val Loss: 0.7653191685676575, Val Accuracy: 73.19999694824219
Iteration 2800, Epoch 4, Loss: 0.8577508926391602, Accuracy: 69.76889038085938, Val Loss: 0.7625284790992737, Val Accuracy: 73.0999984741211
Iteration 3500, Epoch 5, Loss: 0.8031859397888184, Accuracy: 71.64259338378906, Val Loss: 0.7475651502609253, Val Accuracy: 73.29999542236328
Iteration 4200, Epoch 6, Loss: 0.7546246647834778, Accuracy: 73.3280029296875, Val Loss: 0.6867348551750183, Val Accuracy: 76.4000015258789
Iteration 4900, Epoch 7, Lo

Опишите все эксперименты, результаты. Сделайте выводы.

# 1. Использование Keras Model API

Была реализована нейросеть на основе tf.keras.Model (subclassing API).
Основные шаги:

 - объявление слоёв в __init__,

 - реализация прямого прохода в методе call,

 - ручной контроль обучения с помощью tf.GradientTape().

## Выводы:

API tf.keras.Model предоставляет гибкость при создании нестандартных архитектур.

Такой подход требует более подробного описания механики обучения, однако он удобен для построения исследований и кастомных моделей.

Реализация собственных классов моделей помогает глубже понять процесс прямого и обратного проходов.

# 2. Использование Keras Sequential + Functional API

В следующем этапе были протестированы две альтернативные парадигмы построения моделей:

 - Sequential API — простой способ собирать модели послойно.

 - Functional API — даёт больше гибкости: поддерживает произвольные графы вычислений, разветвления, skip-connections.

## Выводы:

Sequential отлично подходит для линейных архитектур и быстрых прототипов.

Functional API является оптимальным средством для сложных сетей, сохраняя при этом удобство декларативного описания.

Обе парадигмы упрощают процесс обучения за счёт встроенных функций .compile() и .fit(), но при этом менее гибкие по сравнению с subclassing API.

### В ходе выполнения работы были освоены три подхода к построению моделей в TensorFlow 2.x. Каждый из них полезен в разных задачах:

| Подход                    | Преимущества                                          | Ограничения                              |
| ------------------------- | ----------------------------------------------------- | ---------------------------------------- |
| **Model Subclassing API** | Максимальная гибкость, подходит для исследований      | Требует больше кода, сложнее дебаг       |
| **Sequential API**        | Самый простой интерфейс, подходит для базовых CNN/MLP | Поддерживает только линейные архитектуры |
| **Functional API**        | Гибкость, простота, декларативность                   | Чуть сложнее, чем Sequential             |
